<a href="https://colab.research.google.com/github/YusunPark/move_bike/blob/main/%EC%BA%A1%EC%8A%A4%ED%86%A4_shortest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import time
import requests
import json
import pandas as pd
import numpy as np
import datetime

In [2]:
# 자전거 데이터 호출
def call_bike():
  BIKE_API_KEY = '74796e6d72736f6c39314a50736843'

  bike_url_1 = f'http://openapi.seoul.go.kr:8088/{BIKE_API_KEY}/json/bikeList/1/1000/'
  bike_url_2 = f'http://openapi.seoul.go.kr:8088/{BIKE_API_KEY}/json/bikeList/1001/2000/'
  bike_url_3 = f'http://openapi.seoul.go.kr:8088/{BIKE_API_KEY}/json/bikeList/2001/3000/'

  resp_bike_1 = requests.get(bike_url_1)
  bike_json_1 = json.loads(resp_bike_1.text)

  resp_bike_2 = requests.get(bike_url_2)
  bike_json_2 = json.loads(resp_bike_2.text)

  resp_bike_3 = requests.get(bike_url_3)
  bike_json_3 = json.loads(resp_bike_3.text)

  df1 = pd.DataFrame.from_dict(bike_json_1['rentBikeStatus']['row'])
  df2 = pd.DataFrame.from_dict(bike_json_2['rentBikeStatus']['row'])
  df3 = pd.DataFrame.from_dict(bike_json_3['rentBikeStatus']['row'])

  return pd.concat([df1, df2, df3])


In [3]:
data = call_bike()

In [4]:
data = data[['stationLatitude','stationLongitude','stationId']]
data.head()

,stationLatitude,stationLongitude,stationId
0,37.55564880,126.91062927,ST-4
1,37.55495071,126.91083527,ST-5
2,37.55062866,126.91498566,ST-6
3,37.55000687,126.91482544,ST-7
4,37.54864502,126.91282654,ST-8


In [11]:
data.iloc[0]

stationLatitude      37.55564880
stationLongitude    126.91062927
stationId                   ST-4
Name: 0, dtype: object

In [56]:
a = pd.DataFrame

In [59]:
a['a'] = distance

TypeError: ignored

In [28]:
API_KEY = '5d1b586339b7fbdfcd3c1088fc54a215'

# 대형마트, 편의점, 학교, 학원, 지하철역, 문화시설
category_group_code = ['MT1', 'CS2', 'SC4', 'AC5', 'SW8', 'CT1']



30분?

In [ ]:
for code in category_group_code[1:]:
  print(f'-------{code}-------')
  # 새 distance 생성
  distance = []
  for i in range(2595):

    bike_stop_y = data.iloc[i][0]
    bike_stop_x = data.iloc[i][1]

    url = f"https://dapi.kakao.com/v2/local/search/category.json?category_group_code={code}&y={bike_stop_y}&x={bike_stop_x}&sort=distance"
    url_result = requests.get(url, params = {'category_group_code': code, 'page': 1, 'size' : 1}, headers={'Authorization' : 'KakaoAK ' + API_KEY}).json()
    distance.append(int(url_result['documents'][0]['distance']))

  data[code] = distance

-------CS2-------
-------SC4-------


In [68]:
len(distance)

522

In [71]:
data


,stationLatitude,stationLongitude,stationId,MT1
0,37.55564880,126.91062927,ST-4,540
1,37.55495071,126.91083527,ST-5,463
2,37.55062866,126.91498566,ST-6,164
3,37.55000687,126.91482544,ST-7,194
4,37.54864502,126.91282654,ST-8,294
...,...,...,...,...
590,37.64805984,127.06239319,ST-3015,132
591,37.50208282,126.89318848,ST-3040,269
592,37.50300598,126.89382935,ST-3041,193
593,37.57164001,127.03565979,ST-1747,431


In [52]:
url_one_point_result

{'documents': [{'address_name': '서울 마포구 서교동 441-17',
   'category_group_code': 'CS2',
   'category_group_name': '편의점',
   'category_name': '가정,생활 > 편의점 > CU',
   'distance': '40',
   'id': '8625621',
   'phone': '02-323-1779',
   'place_name': 'CU 마포제일점',
   'place_url': 'http://place.map.kakao.com/8625621',
   'road_address_name': '서울 마포구 월드컵로14길 5',
   'x': '126.910687444416',
   'y': '37.5560146307157'}],
 'meta': {'is_end': False,
  'pageable_count': 45,
  'same_name': None,
  'total_count': 53858}}

In [54]:
bike_stop_y = data.iloc[0][0]
bike_stop_y

'37.55564880'

In [ ]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

import matplotlib
import matplotlib.pyplot as plt  # 파이플롯 사용
from IPython.display import set_matplotlib_formats
import seaborn as sns
sns.set_style('whitegrid')


set_matplotlib_formats('retina')  # 한글코드를 더 선명하게 해주는 조치, 레티나 설정
matplotlib.rc('font', family='AppleGothic') # 폰트 설정
matplotlib.rc('axes', unicode_minus=False) # 마이너스 폰트가 깨지는 경우가 있으므로 조치

## 관악구 찢어 나누기

In [ ]:
import requests
import json

#geo_path = 'https://raw.githubusercontent.com/TaewoongKong/code_sharing/master/seoul_municipalities_geo_simple.json'
geo_json_file_url = 'https://raw.githubusercontent.com/TaewoongKong/code_sharing/master/seoul_municipalities_geo_simple.json'
resp = requests.get(geo_json_file_url)
geo_json = json.loads(resp.text)
#geo_json = json.load(open(geo_path, encoding='utf-8'))
# print(geo_json)

### gu_code_dict 에 서울시 '구'를 코드로 집어 넣는다.

In [ ]:
gu_code_dict = {}
for num in range(25):
    gu_code_dict[geo_json['features'][num]['properties']['name']] = num
print(gu_code_dict)

{'강동구': 0, '송파구': 1, '강남구': 2, '서초구': 3, '관악구': 4, '동작구': 5, '영등포구': 6, '금천구': 7, '구로구': 8, '강서구': 9, '양천구': 10, '마포구': 11, '서대문구': 12, '은평구': 13, '노원구': 14, '도봉구': 15, '강북구': 16, '성북구': 17, '중랑구': 18, '동대문구': 19, '광진구': 20, '성동구': 21, '용산구': 22, '중구': 23, '종로구': 24}


## 각종 요소 지리 정보 가져오기

- 스타벅스 (추가 1점)
- 맥도날드
- 버거킹
- 다이소 (추가 1점)
- 올리브영
- 지하철역 (추가 2점)
- 편의점

In [ ]:
def grading(target, where_am_i, three_point, two_point, one_point, bonus_point):
    
    apikey = "5d1b586339b7fbdfcd3c1088fc54a215"
    import json
    import requests
    if target == "편의점":
        
        category_group_code = "CS2"
        where_am_i_x = where_am_i[0]
        where_am_i_y = where_am_i[1]
        
        
        url_three_point = f"https://dapi.kakao.com/v2/local/search/category.json?category_group_code={category_group_code}&y={where_am_i_y}&x={where_am_i_x}&radius={three_point}"
        url_three_point_result = requests.get(url_three_point, params = {'category_group_code': category_group_code, 'page': 1, 'size' : 2}, headers={'Authorization' : 'KakaoAK ' + apikey}).json()

        url_two_point = f"https://dapi.kakao.com/v2/local/search/category.json?category_group_code={category_group_code}&y={where_am_i_y}&x={where_am_i_x}&radius={two_point}"
        url_two_point_result = requests.get(url_two_point, params = {'category_group_code': category_group_code, 'page': 1, 'size' : 2} , headers={'Authorization' : 'KakaoAK ' + apikey}).json()

        url_one_point = f"https://dapi.kakao.com/v2/local/search/category.json?category_group_code={category_group_code}&y={where_am_i_y}&x={where_am_i_x}&radius={one_point}"
        url_one_point_result = requests.get(url_one_point, params = {'category_group_code': category_group_code, 'page': 1, 'size' : 2}, headers={'Authorization' : 'KakaoAK ' + apikey}).json()
        
        if len(url_three_point_result['documents']) > 0:
            grade = 3
            is_it_close = f"{three_point}미터 안에 {len(url_three_point_result['documents'])}개 존재"
        elif len(url_two_point_result['documents']) > 0:
            grade = 2
            is_it_close = f"{two_point}미터 안에 {len(url_two_point_result['documents'])}개 존재"
        elif len(url_one_point_result['documents']) > 0:
            grade = 1
            is_it_close = f"{one_point}미터 안에 {len(url_one_point_result['documents'])}개 존재"
        else:
            grade = 0
            is_it_close = f"{one_point}미터 안에 없음"
        
        if bonus_point != 0:
            if grade != 0:
                grade = grade + bonus_point
        
         
        
        return [grade, is_it_close]
        
    elif target == "지하철역":
        
        category_group_code = "SW8"
        where_am_i_x = where_am_i[0]
        where_am_i_y = where_am_i[1]
        
        
        url_three_point = f"https://dapi.kakao.com/v2/local/search/category.json?category_group_code={category_group_code}&y={where_am_i_y}&x={where_am_i_x}&radius={three_point}"
        url_three_point_result = requests.get(url_three_point, params = {'category_group_code': category_group_code, 'page': 1, 'size' : 3}, headers={'Authorization' : 'KakaoAK ' + apikey}).json()

        url_two_point = f"https://dapi.kakao.com/v2/local/search/category.json?category_group_code={category_group_code}&y={where_am_i_y}&x={where_am_i_x}&radius={two_point}"
        url_two_point_result = requests.get(url_two_point, params = {'category_group_code': category_group_code, 'page': 1, 'size' : 3}, headers={'Authorization' : 'KakaoAK ' + apikey}).json()

        url_one_point = f"https://dapi.kakao.com/v2/local/search/category.json?category_group_code={category_group_code}&y={where_am_i_y}&x={where_am_i_x}&radius={one_point}"
        url_one_point_result = requests.get(url_one_point, params = {'category_group_code': category_group_code, 'page': 1, 'size' : 3}, headers={'Authorization' : 'KakaoAK ' + apikey}).json()
        
        if len(url_three_point_result['documents']) > 0:
            grade = 3
            is_it_close = f"{three_point}미터 안에 {len(url_three_point_result['documents'])}개 존재"
        elif len(url_two_point_result['documents']) > 0:
            grade = 2
            is_it_close = f"{two_point}미터 안에 {len(url_two_point_result['documents'])}개 존재"
        elif len(url_one_point_result['documents']) > 0:
            grade = 1
            is_it_close = f"{one_point}미터 안에 {len(url_one_point_result['documents'])}개 존재"
        else:
            grade = 0
            is_it_close = f"{one_point}미터 안에 없음"
        
        if bonus_point != 0:
            if grade != 0:
                grade = grade + bonus_point
        
        return [grade, is_it_close]
        
    else:
        query = target
        
        where_am_i_x = where_am_i[0]
        where_am_i_y = where_am_i[1]
        
        
        url_three_point = f"https://dapi.kakao.com/v2/local/search/keyword.json?query={query}&y={where_am_i_y}&x={where_am_i_x}&radius={three_point}"
        url_three_point_result = requests.get(url_three_point, params = {'query':query, 'page':1, 'size' : 2}, headers={'Authorization' : 'KakaoAK ' + apikey}).json()
        
        url_two_point = f"https://dapi.kakao.com/v2/local/search/keyword.json?query={query}&y={where_am_i_y}&x={where_am_i_x}&radius={two_point}"
        url_two_point_result = requests.get(url_two_point, params = {'query':query, 'page':1, 'size' : 2}, headers={'Authorization' : 'KakaoAK ' + apikey}).json()
        
        url_one_point = f"https://dapi.kakao.com/v2/local/search/keyword.json?query={query}&y={where_am_i_y}&x={where_am_i_x}&radius={one_point}"
        url_one_point_result = requests.get(url_one_point, params = {'query':query, 'page':1, 'size' : 2}, headers={'Authorization' : 'KakaoAK ' + apikey}).json()
        
        if len(url_three_point_result['documents']) > 0:
            grade = 3
            is_it_close = f"{three_point}미터 안에 {len(url_three_point_result['documents'])}개 존재"
        elif len(url_two_point_result['documents']) > 0:
            grade = 2
            is_it_close = f"{two_point}미터 안에 {len(url_two_point_result['documents'])}개 존재"
        elif len(url_one_point_result['documents']) > 0:
            grade = 1
            is_it_close = f"{one_point}미터 안에 {len(url_one_point_result['documents'])}개 존재"
        else:
            grade = 0
            is_it_close = f"{one_point}미터 안에 없음"
        
        if bonus_point != 0:
            if grade != 0:
                grade = grade + bonus_point
        
        return [grade, is_it_close]
        

def final_grading(center_point_list):
    apikey = "5d1b586339b7fbdfcd3c1088fc54a215"
    import json
    import requests
    
    total_grade_list = []
    point_grade_dict_list = []
    dong_name_list = []
    for num in range(len(center_point_list)):
        
        dong_url = f"https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x={center_point_list[num][0]}&y={center_point_list[num][1]}"
        dong_url_result = requests.get(dong_url, headers={'Authorization' : 'KakaoAK ' + apikey}).json()
        dong_name = dong_url_result['documents'][0]['region_3depth_name']
        
        
        point_grade_dict = {}
        starbucks = grading("스타벅스", center_point_list[num], three_point = 350, two_point = 600, one_point = 900, bonus_point = 0)
        macdonald = grading("맥도날드", center_point_list[num], three_point = 350, two_point = 600, one_point = 900, bonus_point = 0)
        burgerking = grading("버거킹", center_point_list[num], three_point = 350, two_point = 600, one_point = 900, bonus_point = 0)
        daiso = grading("다이소", center_point_list[num], three_point = 350, two_point = 600, one_point = 900, bonus_point = 1)
        oliveyoung = grading("올리브영", center_point_list[num], three_point = 350, two_point = 600, one_point = 900, bonus_point = 0)
        subwaystation = grading("지하철역", center_point_list[num], three_point = 350, two_point = 600, one_point = 900, bonus_point = 2)
        cvs = grading("편의점", center_point_list[num], three_point = 200, two_point = 400, one_point = 600, bonus_point = 0)
        
        total_grade_list.append(sum([starbucks[0], macdonald[0], burgerking[0], daiso[0], oliveyoung[0], subwaystation[0], cvs[0]]))

        
        point_grade_dict["스타벅스"] = starbucks[1]
        point_grade_dict["맥도날드"] = macdonald[1]
        point_grade_dict["버거킹"] = burgerking[1]
        point_grade_dict["다이소"] = daiso[1]
        point_grade_dict["올리브영"] = oliveyoung[1]
        point_grade_dict["지하철역"] = subwaystation[1]
        point_grade_dict["편의점"] = cvs[1]
        
        point_grade_dict_list.append(point_grade_dict)
        dong_name_list.append(dong_name)
    return [total_grade_list, point_grade_dict_list, dong_name_list]
        
        
        

In [ ]:
def dividing_gu(gu_name, size):
    
    # 구 코드 정리
    gu_code_dict = {}
    for num in range(25):
        gu_code_dict[geo_json['features'][num]['properties']['name']] = num
    
    gu_code = gu_code_dict[gu_name]
    # 경도 (좌 - 우)
    left_lon = sorted(np.array(geo_json['features'][gu_code]['geometry']['coordinates'][0])[:,0].tolist())[0]  # 가장 작은 경도값 - 제일 왼쪽
    right_lon = sorted(np.array(geo_json['features'][gu_code]['geometry']['coordinates'][0])[:,0].tolist())[-1]  # 가장 큰 경도값 - 제일 오른쪽
    gap_lon = ((right_lon - left_lon) / size)

    lon_list = []
    for num in range(size+1):
        lon_list.append(left_lon + gap_lon*num)
    
    lon_center_list = [] # 중점의 세로 좌표
    for num in range(size):
        lon_center_list.append(np.mean([lon_list[num], lon_list[num + 1]]))

    
    # 위도 (상 - 하)
    top_lat = sorted(np.array(geo_json['features'][gu_code]['geometry']['coordinates'][0])[:,1].tolist())[-1]  # 가장 높은 위도값 - 제일 위쪽
    bottom_lat = sorted(np.array(geo_json['features'][gu_code]['geometry']['coordinates'][0])[:,1].tolist())[0]  # 가장 낮은 위도값 - 제일 위쪽
    gap_lat = ((top_lat - bottom_lat) / size)

    lat_list = []
    for num in range(size + 1):
        lat_list.append(bottom_lat + gap_lat*num)
    
    lat_center_list = []  # 중점의 가로 좌표
    for num in range(size):
        lat_center_list.append(np.mean([lat_list[num], lat_list[num + 1]]))

    # 중점 좌표 모으기
    center_point_list = []
    for lat in lat_center_list:
        for lon in lon_center_list:
            center_point_list.append([lon, lat])
    print(f"{gu_name}의 중점 좌표 모음 = {center_point_list}")
    return center_point_list


In [ ]:
def draw_map(gu_name, size, fianl_grading):
    
    # 구 코드 정리
    gu_code_dict = {}
    for num in range(25):
        gu_code_dict[geo_json['features'][num]['properties']['name']] = num
    
    gu_code = gu_code_dict[gu_name]
    # 경도 (좌 - 우)
    left_lon = sorted(np.array(geo_json['features'][gu_code]['geometry']['coordinates'][0])[:,0].tolist())[0]  # 가장 작은 경도값 - 제일 왼쪽
    right_lon = sorted(np.array(geo_json['features'][gu_code]['geometry']['coordinates'][0])[:,0].tolist())[-1]  # 가장 큰 경도값 - 제일 오른쪽
    gap_lon = ((right_lon - left_lon) / size)

    lon_list = []
    for num in range(size+1):
        lon_list.append(left_lon + gap_lon*num)
    
    lon_center_list = [] # 중점의 세로 좌표
    for num in range(size):
        lon_center_list.append(np.mean([lon_list[num], lon_list[num + 1]]))

    
    # 위도 (상 - 하)
    top_lat = sorted(np.array(geo_json['features'][gu_code]['geometry']['coordinates'][0])[:,1].tolist())[-1]  # 가장 높은 위도값 - 제일 위쪽
    bottom_lat = sorted(np.array(geo_json['features'][gu_code]['geometry']['coordinates'][0])[:,1].tolist())[0]  # 가장 낮은 위도값 - 제일 위쪽
    gap_lat = ((top_lat - bottom_lat) / size)

    lat_list = []
    for num in range(size + 1):
        lat_list.append(bottom_lat + gap_lat*num)
    
    lat_center_list = []  # 중점의 가로 좌표
    for num in range(size):
        lat_center_list.append(np.mean([lat_list[num], lat_list[num + 1]]))

    # 중점 좌표 모으기
    center_point_list = []
    for lat in lat_center_list:
        for lon in lon_center_list:
            center_point_list.append([lon, lat])

            

#     import plotly.graph_objects as go

    fig = go.Figure()

    # 세로선 긋기
    
    
    
    for num in range(size + 1):

        fig.add_trace(go.Scattermapbox(
            showlegend = False,
            mode = "lines",
            lon = [lon_list[num], lon_list[num]], 
            lat = [lat_list[0], lat_list[size]],
            line = {'width' : 1, 'color' : 'gray'}))

    # 가로선 긋기

    for num in range(size + 1):
        fig.add_trace(go.Scattermapbox(
            showlegend = False,
            mode = "lines",
            lon = [lon_list[0], lon_list[size]],
            lat = [lat_list[num], lat_list[num]],
            line = {'width' : 1, 'color' : 'gray'}))

    
    
    # 점 추가하기
    # grading_list 말고 다른 정보까지 리스트로 가져와야한다
                                
    for index, point in enumerate(center_point_list):
        fig.add_trace(go.Scattermapbox(
            showlegend = False,
            mode = "markers",
            lon = [point[0]],
            lat = [point[1]],
            marker_color = 'blue',
            marker_opacity = 0.5,
            marker_size = fianl_grading[0][index],
            # hover text 지정하기
            hoverlabel = dict(bgcolor = 'white'),
            hovertemplate= f"{fianl_grading[2][index]} 인근 자취점수 : {fianl_grading[0][index]}점<br>========<br>맥도날드 : {fianl_grading[1][index]['맥도날드']}<br>버거킹 : {fianl_grading[1][index]['버거킹']}<br>스타벅스 : {fianl_grading[1][index]['스타벅스']}<br>지하철역 : {fianl_grading[1][index]['지하철역']}<br>다이소 : {fianl_grading[1][index]['다이소']}<br>올리브영 : {fianl_grading[1][index]['올리브영']}<br>편의점 : {fianl_grading[1][index]['편의점']}"
        ))
    
    
    
    
    fig.update_layout(
        mapbox = {
            'style': "stamen-terrain",
            'center': {'lon': np.mean(np.array(center_point_list)[:,0]),
                       'lat': np.mean(np.array(center_point_list)[:,1])},
            'zoom': 12,
            'layers': [{'source': np.array(geo_json['features'][gu_code]),
                        'type': "fill",
                        'below': "traces",
                        'color': "royalblue",
                        'opacity': 0.3}]},
        margin = {'l':0, 'r':0, 'b':0, 't':0})
    
 
#     from chart_studio.plotly import plot, iplot

    # 다 만들고 그래프를 fig 와 같은 변수에 담음

    plot(fig, filename = f"{gu_name} - {size}")
    fig.show()

In [ ]:
# pip install tqdm
# pip install plotly --upgrade

In [ ]:
# 준비됐으면 돌리세요!
from tqdm import notebook

for key in notebook.tqdm(list(gu_code_dict.keys())):
# for key in list(gu_code_dict.keys()):
    draw_map(key, 9, final_grading(dividing_gu(key, 9)))#     
#     final_grading(dividing_gu(key, 9))

강동구의 중점 좌표 모음 = [[127.11577405196287, 37.51769886651482], [127.12396931516703, 37.51769886651482], [127.13216457837119, 37.51769886651482], [127.14035984157535, 37.51769886651482], [127.1485551047795, 37.51769886651482], [127.15675036798366, 37.51769886651482], [127.16494563118782, 37.51769886651482], [127.17314089439198, 37.51769886651482], [127.18133615759614, 37.51769886651482], [127.11577405196287, 37.52478298593864], [127.12396931516703, 37.52478298593864], [127.13216457837119, 37.52478298593864], [127.14035984157535, 37.52478298593864], [127.1485551047795, 37.52478298593864], [127.15675036798366, 37.52478298593864], [127.16494563118782, 37.52478298593864], [127.17314089439198, 37.52478298593864], [127.18133615759614, 37.52478298593864], [127.11577405196287, 37.531867105362466], [127.12396931516703, 37.531867105362466], [127.13216457837119, 37.531867105362466], [127.14035984157535, 37.531867105362466], [127.1485551047795, 37.531867105362466], [127.15675036798366, 37.53186710536246

ModuleNotFoundError: No module named 'plotly'

In [ ]:
a_list = list(gu_code_dict.keys())
a_list.sort()
a_list
